# Monarch

One of Monarch's more powerful features is its Actor/endpoint API, which provides a generic interface for distributed computing. In this notebook, we introduce some of the basics

## Hello World
Actors are spawned in Process meshes via the `monarch.proc_mesh` API. For those familiar with distributed systems, it can be helpful to think of each Actor as a server with endpoints that can be called.

In [ ]:
import asyncio

from monarch.proc_mesh import proc_mesh, ProcMesh
from monarch.service import Actor, endpoint, current_rank

NUM_ACTORS=4

class ToyActor(Actor):
    def __init__(self):
        self.rank = current_rank().rank
    
    @endpoint
    async def hello_world(self, msg):
        print(f"Identity: {self.rank}, {msg=}")        

# Note: Meshes can be also be created on different nodes, but we're ignoring that in this example
local_proc_mesh = await proc_mesh(gpus=NUM_ACTORS)
# This spawns 4 instances of 'ToyActor'
toy_actor = await local_proc_mesh.spawn("toy_actor", ToyActor) 

In [ ]:
# Once actors are spawned, we can call all of them simultaneiously with `Actor.endpoint.call` as below
await toy_actor.hello_world.call("hey there, from jupyter!!")

In [ ]:
# We can also specify a single actor using the 'slice' API
futures = []
for idx in range(NUM_ACTORS):
    actor_instance = toy_actor.slice(gpus=idx)
    futures.append(actor_instance.hello_world.call_one(f"Here's an arbitrary unique value: {idx}"))

# conveniently, we can still schedule & gather them in parallel using asyncio
await asyncio.gather(*futures)

## Ping Pong
Not only is it possible to call endpoints froma 'main' fuction, but actors have the useful property of being able to communicate with one another. 

In [ ]:
import asyncio

from monarch.proc_mesh import proc_mesh, ProcMesh
from monarch.service import Actor, endpoint, current_rank

class ExampleActor(Actor):
    def __init__(self, actor_name):
        self.actor_name=actor_name
    
    @endpoint
    async def init(self, other_actor):
        self.other_actor = other_actor
        self.other_actor_pair = other_actor.slice(**current_rank())
        self.identity = current_rank().rank
    
    @endpoint
    async def send(self, msg):
        self.other_actor_pair.recv.call(f"Sender ({self.actor_name}:{self.identity}) {msg=}")
        
    @endpoint
    async def recv(self, msg):
        print(f"Pong!, Receiver ({self.actor_name}:{self.identity}) received msg {msg}")

# Spawn two different Actors in different meshes, with two instances each
local_mesh_0 = await proc_mesh(gpus=2)
actor_0 = await local_mesh_0.spawn(
    "actor_0",
    ExampleActor,
    "actor_0"     # this arg is passed to ExampleActor.__init__
) 

local_mesh_1 = await proc_mesh(gpus=2)
actor_1 = await local_mesh_1.spawn(
    "actor_1",
    ExampleActor,
    "actor_1"     # this arg is passed to ExampleActor.__init__
) 

In [ ]:
# Initialize each actor with references to each other
asyncio.gather(
    actor_0.init.call(actor_1),
    actor_1.init.call(actor_0),
)

In [ ]:
await actor_0.send.call("Ping")

In [ ]:
await actor_1.send.call("Ping")